## Packages

In [ ]:
import pandas as pd
import numpy as np
## import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import sqlalchemy 
from sqlalchemy import create_engine, text

import sys
import os

## Add the path of the functions folder
current_dir = os.getcwd()  ## Gets the current working directory
sub_dir = os.path.abspath(os.path.join(current_dir, '..'
                                       , 'Functions'))
sys.path.append(sub_dir)

# Now you can import functions
from db_secrets import SQL_107

#from visualisations import plot_prediction_error, plot_prediction_density_subplots

from helpers import aggregate_sites

In [ ]:
# scikit-survival
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.preprocessing import OneHotEncoder
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest

from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from datetime import datetime

## Connection

In [ ]:
## text for query
with open("../Exploratory_Analysis/111_sql.sql", "r") as file:
    query_text = file.read()

query_text = query_text.replace('REPLACE START DATE','2022-01-01')

In [ ]:
## Create an engine + connection
engine = create_engine(SQL_107())
conn = engine.connect()

## Return data
df_raw = pd.read_sql(query_text,conn)

## Wrangle

In [ ]:
## Makes working copy
df = df_raw.copy()

#df = df.sample(n=100000, random_state=42)

In [ ]:
## List columns
df.columns

In [ ]:
df = df[['Call Connect Time'
         ,'Outcome Location Name'
         ,'Bank Holiday'
         , 'In_Out_Hours'
         , 'Sub ICB Name'
         ,'Outcome Type'
         ,'Outcome Datetime']].copy()

In [ ]:
df['Outcome'] = df['Outcome Type'].transform(lambda x: False if x == 'No UEC Contact' else True)
df = df.drop(['Outcome Type'],axis=1) 

In [ ]:
## Round time to nearest minute
df['Call Connect Time'] = df['Call Connect Time'].dt.round(freq='min')
df['Outcome Datetime'] = df['Outcome Datetime'].dt.round(freq='min')


In [ ]:
df['Mins to outcome'] = df['Outcome Datetime'] - df['Call Connect Time']
df['Mins to outcome'] = df['Mins to outcome'].dt.total_seconds()/60

## right censored data upto 24 hours
df['Mins to outcome'] = df['Mins to outcome'].fillna(1441) ## minutes in day+1
df['Mins to outcome'] = df['Mins to outcome'].transform(lambda x: 1441 if x > 1441 else x) 

## removes zeros and less than zero
df = df[df['Mins to outcome'] > 0]

In [ ]:
## Replaces low frequency sites with 'OTHER SITE'
df['Outcome Location Name'] = (df['Outcome Location Name']
                               .apply(lambda x: aggregate_sites(x)))

In [ ]:
## groups rare sites for a place
positive_counts = df.groupby(['Sub ICB Name', 'Outcome Location Name'])['Outcome'].sum().reset_index(name='Attends')

total_positives = positive_counts.groupby('Sub ICB Name')['Attends'].sum().reset_index(name='Total_Attends')

lu_site_agg = positive_counts.merge(total_positives, on='Sub ICB Name')
lu_site_agg['Percentage'] = (lu_site_agg['Attends'] / lu_site_agg['Total_Attends']) * 100

lu_site_agg['Location'] = 'OTHER SITE'

## keep details of sites with > 5% of activity
lu_site_agg.loc[(lu_site_agg['Percentage'] > 5) &
                (lu_site_agg['Outcome Location Name'] != 'No UEC Contact')
                , 'Location'] = lu_site_agg['Outcome Location Name']

lu_site_agg.loc[lu_site_agg['Outcome Location Name'] == 'No UEC Contact'
                , 'Location'] = 'No UEC Contact'


In [ ]:
## Add new location
df=pd.merge(df
         ,lu_site_agg[['Sub ICB Name','Outcome Location Name','Location']]
         , on = ['Sub ICB Name','Outcome Location Name']
         , how='left')

## Drop previous location
df = df.drop('Outcome Location Name', axis=1)

### date time

In [ ]:
## Date time conversion to numeric
df['year']    = df['Call Connect Time'].dt.year

df['month sin'] = np.sin(df['Call Connect Time'].dt.month * (2*np.pi/12))
df['month cos'] = np.cos(df['Call Connect Time'].dt.month * (2*np.pi/12))

df['YearDay sin'] = np.sin(df['Call Connect Time'].dt.day_of_year * (2*np.pi/365))
df['YearDay cos'] = np.cos(df['Call Connect Time'].dt.day_of_year * (2*np.pi/365))

df['weekday sin'] = np.sin(df['Call Connect Time'].dt.weekday+1 * (2*np.pi/7))  # Monday=0, Sunday=6
df['weekday cos'] = np.cos(df['Call Connect Time'].dt.weekday+1 * (2*np.pi/7))  # Monday=0, Sunday=6

df['Hour sin'] = np.sin(df['Call Connect Time'].dt.hour * (2*np.pi/24))
df['Hour cos'] = np.cos(df['Call Connect Time'].dt.hour * (2*np.pi/24))

df = df.drop('Call Connect Time',axis=1) 
df = df.drop('Outcome Datetime',axis=1) 

In [ ]:
df['Location'] = df['Location'].astype('category')
df['Bank Holiday'] = df['Bank Holiday'].astype('category')
df['In_Out_Hours'] = df['In_Out_Hours'].astype('category')
df['Sub ICB Name'] = df['Sub ICB Name'].astype('category')

## split

In [ ]:
outcome_cols = ['Outcome','Mins to outcome']

X = df.drop(outcome_cols,axis=1)
y = df[outcome_cols]

y = np.array(
    list(y.itertuples(index=False, name=None)),  # Convert rows to tuples
    dtype=[('Outcome', '?'), ('Mins to outcome', '<f8')]  # Define the structured dtype
    )


X_train, X_test, y_train, y_test = train_test_split(X
                                                    , y 
                                                    , stratify=y["Outcome"] ## make sure there are equal proportions in test and train
                                                    , test_size = 0.25
                                                    , random_state=42)

## kaplan_meier_estimator

In [ ]:
time, survival_prob, conf_int = kaplan_meier_estimator(
          y_train["Outcome"]
        , y_train["Mins to outcome"]
        , conf_type="log-log"
        )

plt.step(time, survival_prob, where="post")
plt.fill_between(time, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.ylim(0, 1)
plt.ylabel(r"est. probability of not attend UEC $\hat{S}(t)$")
plt.xlabel("minutes $t$")


In [ ]:
place_list = []
place_list = df['Sub ICB Name'].unique()

In [ ]:
place_list

In [ ]:
for place in place_list:
    mask_place = X_train["Sub ICB Name"] == place
    time, survival_prob, conf_int = kaplan_meier_estimator(
                                    y_train["Outcome"][mask_place]
                                    , y_train["Mins to outcome"][mask_place],
                                    conf_type="log-log",
                                    )

    plt.step(time, survival_prob, where="post", label=f"Place = {place}")
    plt.fill_between(time, np.nan_to_num(conf_int[0])
                        , np.nan_to_num(conf_int[1])
                        , alpha=0.25, step="post")

plt.ylim(0, 1)
plt.ylabel(r"est. probability of not attend UEC $\hat{S}(t)$")
plt.xlabel("minutes $t$")
plt.legend(loc="best")


# Random forest

#### create a new df with one copy of the data per site

In [ ]:

Outcome_Location = df[ ~df['Location'].isin(
                            [ 'No UEC Contact', 'OTHER SITE']) ]['Location'].unique()

new_df = pd.DataFrame()

#for Location in Outcome_Location:
for Location in ['UNIVERSITY HOSPITAL OF NORTH DURHAM']:
    print(Location)
    temp_df = df.copy()
    temp_df['Site Version'] = Location
    temp_df['Outcome'] = temp_df['Location'] == Location
    #temp_df['Mins to outcome'] = np.where(temp_df['Outcome'], temp_df['Mins to outcome'], 1441)

    new_df = pd.concat([new_df,temp_df], ignore_index=True, sort=False)

new_df['Site Version'] = new_df['Site Version'].astype('category')


#### split

In [ ]:
outcome_cols = ['Outcome','Mins to outcome']

new_X = new_df.drop(outcome_cols,axis=1)
new_X = new_X.drop(['Location'],axis=1)
new_y = new_df[outcome_cols]

new_y = np.array(
    list(new_y.itertuples(index=False, name=None)),  # Convert rows to tuples
    dtype=[('Outcome', '?'), ('Mins to outcome', '<f8')]  # Define the structured dtype
    )


X_train, X_test, y_train, y_test = train_test_split(new_X
                                                    , new_y 
                                                    , stratify=new_y["Outcome"] ## make sure there are equal proportions in test and train
                                                    , test_size = 0.25
                                                    , random_state=42)

In [ ]:
#outcome_cols = ['Outcome','Mins to outcome']
#
#X = df.drop(outcome_cols,axis=1)
#y = df[outcome_cols]
#
#y = np.array(
#    list(y.itertuples(index=False, name=None)),  # Convert rows to tuples
#    dtype=[('Outcome', '?'), ('Mins to outcome', '<f8')]  # Define the structured dtype
#    )
#
#
#X_train, X_test, y_train, y_test = train_test_split(X
#                                                    , y 
#                                                    , stratify=y["Outcome"] ## make sure there are equal proportions in test and train
#                                                    , test_size = 0.25
#                                                    , random_state=42)

### fit the model

In [ ]:
#cph = make_pipeline(OneHotEncoder(), CoxPHSurvivalAnalysis())
#cph.fit(X_train, y_train) ##takes 30+ mins


rsf = make_pipeline(OneHotEncoder(), RandomSurvivalForest(n_estimators=100
                                                          , min_samples_leaf=7
                                                          , random_state=42
                                                          ,low_memory=False))
rsf.fit(X_train, y_train)


In [ ]:
arrival_windows = np.arange(20, 241, 5)

rsf_chf_funcs = rsf.predict_cumulative_hazard_function(X_test, return_array=False)
rsf_risk_scores = np.vstack([chf(arrival_windows) for chf in rsf_chf_funcs])

rsf_auc, rsf_mean_auc = cumulative_dynamic_auc(y_train, y_test, rsf_risk_scores, arrival_windows)


In [ ]:
cph_risk_scores = cph.predict(X_test)


In [ ]:
arrival_windows = np.arange(20, 241, 5)
cph_auc, cph_mean_auc = cumulative_dynamic_auc(y_train
                                               , y_test
                                               , cph_risk_scores
                                               , arrival_windows)

plt.plot(arrival_windows, cph_auc, marker="o")
plt.axhline(cph_mean_auc, linestyle="--")
plt.xlabel("minutes from call")
plt.ylabel("time-dependent AUC")
plt.grid(True)

In [ ]:
from sksurv.nonparametric import CensoringDistributionEstimator

# Fit the censoring distribution estimator
cens = CensoringDistributionEstimator()
cens.fit(y_train)

# Predict censoring probabilities for arrival windows
censoring_probs = cens.predict_proba(arrival_windows)

# Print problematic time points
print("Censoring survival probabilities:", censoring_probs)
print("Arrival windows with zero censoring survival function:", arrival_windows[censoring_probs == 0])

# Filter out invalid time points
valid_times = arrival_windows[censoring_probs > 0]

# Recompute AUC with valid time points
cph_auc, cph_mean_auc = cumulative_dynamic_auc(y_train, y_test, cph_risk_scores, valid_times)


In [ ]:
from sksurv.datasets import load_veterans_lung_cancer
va_x, va_y = load_veterans_lung_cancer()


In [ ]:

va_x, va_y = load_veterans_lung_cancer()

va_x_train, va_x_test, va_y_train, va_y_test = train_test_split(
    va_x, va_y, test_size=0.2, stratify=va_y["Status"], random_state=0
)

va_cph = make_pipeline(OneHotEncoder(), CoxPHSurvivalAnalysis())
va_cph.fit(va_x_train, va_y_train)

va_times = np.arange(8, 184, 7)
va_cph_risk_scores = va_cph.predict(va_x_test)
va_cph_auc, va_cph_mean_auc = cumulative_dynamic_auc(va_y_train, va_y_test, va_cph_risk_scores, va_times)

plt.plot(va_times, va_cph_auc, marker="o")
plt.axhline(va_cph_mean_auc, linestyle="--")
plt.xlabel("days from enrollment")
plt.ylabel("time-dependent AUC")
plt.grid(True)


In [ ]:
va_cph_risk_scores